# Importing Libraries

In [1]:
# Importing pre-trained model
from keras.applications import ResNet50V2

# Importing important layers of Keras
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import EarlyStopping

# Importing data functions
from data_library.data_processing import black_white, crop, delete_face_images, encode, to_cat

# Importing preprocessing functions
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


# Importing the pre-trained model

In [2]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

# Saving the model in a variable
resnet = ResNet50V2(weights='imagenet', include_top=False)


# 10 Classes from Real Madrid (cropped_faces)

In [3]:
faces_folder = "../data_processing/raw_data/faces/"
cropped_faces_directory = "../data_processing/raw_data/cropped_faces"


In [4]:
# # RUN ONLY IF YOU DONT HAVE THE CROPPED FACES YET
# black_white(faces_folder)
# crop(faces_folder, cropped_faces_directory)
# delete_face_images(cropped_faces_directory)


In [5]:
y, X = encode(cropped_faces_directory)
y_cat = to_cat(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)


## Model 1

In [6]:
# Add a new classification head
layer = resnet.output
layer = GlobalAveragePooling2D()(layer)
predictions = Dense(y_train.shape[1], activation='softmax')(layer)

model = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

# Train the model
model.fit(X_train,
          y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[es]
         )


Epoch 1/10


2023-11-25 14:17:47.472006: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


10/10 [==============================] - 5s 224ms/step - loss: 2.3223 - accuracy: 0.2564 - val_loss: 23.9073 - val_accuracy: 0.0500
Epoch 2/10
10/10 [==============================] - 2s 167ms/step - loss: 1.3502 - accuracy: 0.5064 - val_loss: 60.0865 - val_accuracy: 0.1000
Epoch 3/10
10/10 [==============================] - 2s 176ms/step - loss: 0.8245 - accuracy: 0.7885 - val_loss: 39.2152 - val_accuracy: 0.0750
Epoch 4/10
10/10 [==============================] - 2s 181ms/step - loss: 0.7260 - accuracy: 0.8077 - val_loss: 23.0652 - val_accuracy: 0.1250
Epoch 5/10
10/10 [==============================] - 2s 185ms/step - loss: 0.5079 - accuracy: 0.8397 - val_loss: 21.9407 - val_accuracy: 0.2000
Epoch 6/10
10/10 [==============================] - 2s 193ms/step - loss: 0.2986 - accuracy: 0.9167 - val_loss: 18.1469 - val_accuracy: 0.2000
Epoch 7/10
10/10 [==============================] - 2s 187ms/step - loss: 0.1976 - accuracy: 0.9423 - val_loss: 12.2690 - val_accuracy: 0.2000
Epoch 8/10

In [7]:
model.evaluate(X_test, y_test)


2/2 [==============================] - 0s 41ms/step - loss: 10.4259 - accuracy: 0.2041


[10.425868034362793, 0.20408163964748383]

## Model 2

In [8]:
# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add augmentation layers directly within the model
augmented = Conv2D(32, (3, 3), activation='relu')(input_layer)
augmented = MaxPooling2D((2, 2))(augmented)
# Add more layers or augmentation techniques as needed

# Rest of your model architecture
# Example:
augmented = Flatten()(augmented)
augmented = Dense(128, activation='relu')(augmented)
output_layer = Dense(y_train.shape[1], activation='softmax')(augmented)


model_2 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

datagen = ImageDataGenerator(
    rotation_range=40,       # Rotate images by a wider range of degrees
    width_shift_range=0.3,   # Shift images horizontally by a larger fraction of total width
    height_shift_range=0.3,  # Shift images vertically by a larger fraction of total height
    shear_range=0.3,         # Apply a larger shear-based transformation
    zoom_range=0.3,          # Zoom in or out of images more aggressively
    horizontal_flip=True,    # Flip images horizontally
    vertical_flip=True,      # Flip images vertically
    fill_mode='nearest'      # Fill points outside the boundaries using the nearest available value
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_2.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
7/7 [==============================] - 0s 15ms/step - loss: 502.6071 - accuracy: 0.1582
Epoch 2/100
7/7 [==============================] - 0s 15ms/step - loss: 342.1305 - accuracy: 0.1122
Epoch 3/100
7/7 [==============================] - 0s 15ms/step - loss: 91.2219 - accuracy: 0.1224
Epoch 4/100
7/7 [==============================] - 0s 15ms/step - loss: 14.4351 - accuracy: 0.1327
Epoch 5/100
7/7 [==============================] - 0s 17ms/step - loss: 5.2588 - accuracy: 0.0867
Epoch 6/100
7/7 [==============================] - 0s 15ms/step - loss: 3.1977 - accuracy: 0.1173
Epoch 7/100
7/7 [==============================] - 0s 14ms/step - loss: 2.8643 - accuracy: 0.1531
Epoch 8/100
7/7 [==============================] - 0s 15ms/step - loss: 2.5858 - accuracy: 0.1429
Epoch 9/100
7/7 [==============================] - 0s 15ms/step - loss: 2.6589 - accuracy: 0.1480
Epoch 10/100
7/7 [==============================] - 0s 15ms/step - loss: 2.6923 - accuracy: 0.1276
Epoch 11/100


In [9]:
model_2.evaluate(X_test, y_test)


2/2 [==============================] - 0s 4ms/step - loss: 2.3625 - accuracy: 0.2653


[2.3625125885009766, 0.26530611515045166]

## Model 3

In [10]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add convolutional layers with pooling
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

# Flatten the output before feeding into dense layers
flatten = Flatten()(pool2)

# Add dense layers with dropout for regularization
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)  # Dropout layer for regularization

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)  # Dropout layer for regularization

# Output layer with softmax activation for multi-class classification
output_layer = Dense(y_train.shape[1], activation='softmax')(dropout2)

# Create the model
model_3 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create an ImageDataGenerator with multiple augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Vary brightness
    channel_shift_range=50,        # Vary channel shifts
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_3.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
7/7 [==============================] - 0s 22ms/step - loss: 123.3358 - accuracy: 0.1173
Epoch 2/100
7/7 [==============================] - 0s 22ms/step - loss: 10.3514 - accuracy: 0.0918
Epoch 3/100
7/7 [==============================] - 0s 21ms/step - loss: 2.6886 - accuracy: 0.0867
Epoch 4/100
7/7 [==============================] - 0s 23ms/step - loss: 2.3335 - accuracy: 0.0969
Epoch 5/100
7/7 [==============================] - 0s 20ms/step - loss: 2.3457 - accuracy: 0.0969
Epoch 6/100
7/7 [==============================] - 0s 22ms/step - loss: 2.3029 - accuracy: 0.0714
Epoch 7/100
7/7 [==============================] - 0s 21ms/step - loss: 2.3216 - accuracy: 0.0969
Epoch 8/100
7/7 [==============================] - 0s 19ms/step - loss: 2.3272 - accuracy: 0.0969
Epoch 9/100
7/7 [==============================] - 0s 22ms/step - loss: 2.3052 - accuracy: 0.1224
Epoch 10/100
7/7 [==============================] - 0s 22ms/step - loss: 2.3560 - accuracy: 0.1020
Epoch 11/100
7/7

In [11]:
model_3.evaluate(X_test, y_test)


2/2 [==============================] - 0s 5ms/step - loss: 2.3279 - accuracy: 0.0816


[2.3279380798339844, 0.08163265138864517]

## Model 4

In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_4 = Sequential()
model_4.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Conv2D(64, (3, 3), activation='relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Conv2D(128, (3, 3), activation='relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Flatten())
model_4.add(Dense(256, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_4.add(Dropout(0.5))  # Adding Dropout layer
model_4.add(Dense(128, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_4.add(Dense(y_train.shape[1], activation='softmax'))

# Using Adam optimizer with a reduced learning rate
optimizer = Adam(lr=0.0001)
model_4.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Adding more callbacks, e.g., ReduceLROnPlateau
callbacks = [EarlyStopping(patience=5, restore_best_weights=True),
             ReduceLROnPlateau(factor=0.1, patience=3)]

# Fit the model with increased complexity and callbacks
history = model_4.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=(X_test, y_test))

model_4.evaluate(X_test, y_test)


Epoch 1/100


/Users/yassinebouaine/.pyenv/versions/3.10.6/envs/dynamic-players-insights/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


13/13 [==============================] - 1s 25ms/step - loss: 20.3172 - accuracy: 0.1224 - val_loss: 7.4262 - val_accuracy: 0.0816 - lr: 0.0010
Epoch 2/100
13/13 [==============================] - 0s 17ms/step - loss: 7.1977 - accuracy: 0.1276 - val_loss: 6.8212 - val_accuracy: 0.1224 - lr: 0.0010
Epoch 3/100
13/13 [==============================] - 0s 18ms/step - loss: 6.7182 - accuracy: 0.1122 - val_loss: 6.5903 - val_accuracy: 0.1020 - lr: 0.0010
Epoch 4/100
13/13 [==============================] - 0s 17ms/step - loss: 6.4913 - accuracy: 0.1327 - val_loss: 6.3649 - val_accuracy: 0.2041 - lr: 0.0010
Epoch 5/100
13/13 [==============================] - 0s 18ms/step - loss: 6.3076 - accuracy: 0.1582 - val_loss: 6.2227 - val_accuracy: 0.2245 - lr: 0.0010
Epoch 6/100
13/13 [==============================] - 0s 18ms/step - loss: 6.1535 - accuracy: 0.1480 - val_loss: 6.0806 - val_accuracy: 0.1224 - lr: 0.0010
Epoch 7/100
13/13 [==============================] - 0s 17ms/step - loss: 5.9992 

[3.5108349323272705, 0.36734694242477417]

## Model 5

In [13]:
from keras.layers import BatchNormalization, LeakyReLU

model_5 = Sequential()
model_5.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())  # Add BatchNormalization after each Conv layer
model_5.add(Conv2D(64, (3, 3), activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())
model_5.add(Conv2D(128, (3, 3), activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(BatchNormalization())
model_5.add(Flatten())
model_5.add(Dense(256))
model_5.add(LeakyReLU(alpha=0.1))  # LeakyReLU activation
model_5.add(Dropout(0.5))
model_5.add(Dense(128))
model_5.add(LeakyReLU(alpha=0.1))
model_5.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model_5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with adjusted architecture
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model_5.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=150,  # Train for more epochs
                    callbacks=es,
                    validation_data=(X_test, y_test))

model_5.evaluate(X_test, y_test)


Epoch 1/150
4/4 [==============================] - 1s 107ms/step - loss: 3.8174 - accuracy: 0.1224 - val_loss: 11.8685 - val_accuracy: 0.0612
Epoch 2/150
4/4 [==============================] - 0s 46ms/step - loss: 3.6347 - accuracy: 0.1633 - val_loss: 6.1903 - val_accuracy: 0.1224
Epoch 3/150
4/4 [==============================] - 0s 43ms/step - loss: 3.1312 - accuracy: 0.1684 - val_loss: 5.2107 - val_accuracy: 0.0612
Epoch 4/150
4/4 [==============================] - 0s 54ms/step - loss: 3.3831 - accuracy: 0.1582 - val_loss: 5.4113 - val_accuracy: 0.1224
Epoch 5/150
4/4 [==============================] - 0s 49ms/step - loss: 3.4657 - accuracy: 0.1429 - val_loss: 5.5982 - val_accuracy: 0.0408
Epoch 6/150
4/4 [==============================] - 0s 46ms/step - loss: 3.0794 - accuracy: 0.2041 - val_loss: 5.4814 - val_accuracy: 0.1020
Epoch 7/150
4/4 [==============================] - 0s 48ms/step - loss: 2.9666 - accuracy: 0.2245 - val_loss: 6.3304 - val_accuracy: 0.0816
Epoch 8/150
4/4 [=

[2.275377035140991, 0.2448979616165161]

# Video - 10 Classes (video_cropped_faces_0)

In [14]:
video_cropped_faces_directory = "../data_processing/raw_data/video_cropped_faces_0"


In [58]:
y, X = encode(video_cropped_faces_directory)
y_cat = to_cat(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)


In [16]:
y_train.shape


(63, 11)

## Model 1

In [17]:
# Add a new classification head
layer = resnet.output
layer = GlobalAveragePooling2D()(layer)
predictions = Dense(y_train.shape[1], activation='softmax')(layer)

model_6 = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model_6.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

# Train the model
model_6.fit(X_train,
          y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[es]
         )


Epoch 1/10
4/4 [==============================] - 3s 300ms/step - loss: 2.4580 - accuracy: 0.3200 - val_loss: 5.4721 - val_accuracy: 0.3846
Epoch 2/10
4/4 [==============================] - 1s 176ms/step - loss: 1.2694 - accuracy: 0.7000 - val_loss: 6.5433 - val_accuracy: 0.3077
Epoch 3/10
4/4 [==============================] - 1s 161ms/step - loss: 0.7697 - accuracy: 0.7800 - val_loss: 40.3561 - val_accuracy: 0.0769
Epoch 4/10
4/4 [==============================] - 1s 163ms/step - loss: 0.3518 - accuracy: 0.9000 - val_loss: 48.2298 - val_accuracy: 0.0769


In [18]:
model_6.evaluate(X_test, y_test)


1/1 [==============================] - 0s 74ms/step - loss: 5.3977 - accuracy: 0.4375


[5.397744178771973, 0.4375]

## Model 2

In [19]:
# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add augmentation layers directly within the model
augmented = Conv2D(32, (3, 3), activation='relu')(input_layer)
augmented = MaxPooling2D((2, 2))(augmented)
# Add more layers or augmentation techniques as needed

# Rest of your model architecture
# Example:
augmented = Flatten()(augmented)
augmented = Dense(128, activation='relu')(augmented)
output_layer = Dense(y_train.shape[1], activation='softmax')(augmented)


model_7 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_7.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

datagen = ImageDataGenerator(
    rotation_range=40,       # Rotate images by a wider range of degrees
    width_shift_range=0.3,   # Shift images horizontally by a larger fraction of total width
    height_shift_range=0.3,  # Shift images vertically by a larger fraction of total height
    shear_range=0.3,         # Apply a larger shear-based transformation
    zoom_range=0.3,          # Zoom in or out of images more aggressively
    horizontal_flip=True,    # Flip images horizontally
    vertical_flip=True,      # Flip images vertically
    fill_mode='nearest'      # Fill points outside the boundaries using the nearest available value
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_7.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
2/2 [==============================] - 0s 18ms/step - loss: 367.1548 - accuracy: 0.0794
Epoch 2/100
2/2 [==============================] - 0s 16ms/step - loss: 453.9112 - accuracy: 0.2063
Epoch 3/100
2/2 [==============================] - 0s 17ms/step - loss: 259.1899 - accuracy: 0.1270
Epoch 4/100
2/2 [==============================] - 0s 16ms/step - loss: 171.7191 - accuracy: 0.0952
Epoch 5/100
2/2 [==============================] - 0s 14ms/step - loss: 125.9402 - accuracy: 0.0635
Epoch 6/100
2/2 [==============================] - 0s 16ms/step - loss: 102.1645 - accuracy: 0.2698
Epoch 7/100
2/2 [==============================] - 0s 17ms/step - loss: 85.0283 - accuracy: 0.2698
Epoch 8/100
2/2 [==============================] - 0s 24ms/step - loss: 64.5194 - accuracy: 0.3492
Epoch 9/100
2/2 [==============================] - 0s 15ms/step - loss: 44.7028 - accuracy: 0.3492
Epoch 10/100
2/2 [==============================] - 0s 17ms/step - loss: 40.3336 - accuracy: 0.2222
Epo

In [20]:
model_7.evaluate(X_test, y_test)


1/1 [==============================] - 0s 68ms/step - loss: 1.0431 - accuracy: 0.5625


[1.043073296546936, 0.5625]

## Model 3

In [21]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add convolutional layers with pooling
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

# Flatten the output before feeding into dense layers
flatten = Flatten()(pool2)

# Add dense layers with dropout for regularization
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)  # Dropout layer for regularization

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)  # Dropout layer for regularization

# Output layer with softmax activation for multi-class classification
output_layer = Dense(y_train.shape[1], activation='softmax')(dropout2)

# Create the model
model_8 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_8.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create an ImageDataGenerator with multiple augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Vary brightness
    channel_shift_range=50,        # Vary channel shifts
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_8.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
2/2 [==============================] - 0s 23ms/step - loss: 21.5049 - accuracy: 0.0952
Epoch 2/100
2/2 [==============================] - 0s 25ms/step - loss: 21.8414 - accuracy: 0.0635
Epoch 3/100
2/2 [==============================] - 0s 22ms/step - loss: 9.4386 - accuracy: 0.0952
Epoch 4/100
2/2 [==============================] - 0s 22ms/step - loss: 3.1465 - accuracy: 0.1746
Epoch 5/100
2/2 [==============================] - 0s 19ms/step - loss: 2.7752 - accuracy: 0.0952
Epoch 6/100
2/2 [==============================] - 0s 20ms/step - loss: 2.4116 - accuracy: 0.1905
Epoch 7/100
2/2 [==============================] - 0s 19ms/step - loss: 2.7030 - accuracy: 0.1111
Epoch 8/100
2/2 [==============================] - 0s 20ms/step - loss: 2.3904 - accuracy: 0.0794
Epoch 9/100
2/2 [==============================] - 0s 27ms/step - loss: 2.3126 - accuracy: 0.0952
Epoch 10/100
2/2 [==============================] - 0s 24ms/step - loss: 2.3411 - accuracy: 0.2698
Epoch 11/100
2/2 

In [22]:
model_8.evaluate(X_test, y_test)


1/1 [==============================] - 0s 77ms/step - loss: 1.8340 - accuracy: 0.4375


[1.833991289138794, 0.4375]

## Model 4

In [23]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_9 = Sequential()
model_9.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_9.add(MaxPooling2D((2, 2)))
model_9.add(Conv2D(64, (3, 3), activation='relu'))
model_9.add(MaxPooling2D((2, 2)))
model_9.add(Conv2D(128, (3, 3), activation='relu'))
model_9.add(MaxPooling2D((2, 2)))
model_9.add(Flatten())
model_9.add(Dense(256, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_9.add(Dropout(0.5))  # Adding Dropout layer
model_9.add(Dense(128, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_9.add(Dense(y_train.shape[1], activation='softmax'))

# Using Adam optimizer with a reduced learning rate
optimizer = Adam(lr=0.0001)
model_9.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Adding more callbacks, e.g., ReduceLROnPlateau
callbacks = [EarlyStopping(patience=5, restore_best_weights=True),
             ReduceLROnPlateau(factor=0.1, patience=3)]

# Fit the model with increased complexity and callbacks
history = model_9.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=(X_test, y_test))


Epoch 1/100


/Users/yassinebouaine/.pyenv/versions/3.10.6/envs/dynamic-players-insights/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


4/4 [==============================] - 1s 49ms/step - loss: 21.0827 - accuracy: 0.0794 - val_loss: 12.9664 - val_accuracy: 0.1250 - lr: 0.0010
Epoch 2/100
4/4 [==============================] - 0s 22ms/step - loss: 12.0272 - accuracy: 0.1111 - val_loss: 8.6023 - val_accuracy: 0.0625 - lr: 0.0010
Epoch 3/100
4/4 [==============================] - 0s 20ms/step - loss: 8.8311 - accuracy: 0.1746 - val_loss: 7.5499 - val_accuracy: 0.0625 - lr: 0.0010
Epoch 4/100
4/4 [==============================] - 0s 21ms/step - loss: 7.5964 - accuracy: 0.1111 - val_loss: 7.1713 - val_accuracy: 0.1250 - lr: 0.0010
Epoch 5/100
4/4 [==============================] - 0s 22ms/step - loss: 7.0736 - accuracy: 0.2381 - val_loss: 7.0089 - val_accuracy: 0.0625 - lr: 0.0010
Epoch 6/100
4/4 [==============================] - 0s 20ms/step - loss: 6.9963 - accuracy: 0.2222 - val_loss: 6.8592 - val_accuracy: 0.0625 - lr: 0.0010
Epoch 7/100
4/4 [==============================] - 0s 24ms/step - loss: 6.8719 - accuracy: 

In [24]:
model_9.evaluate(X_test, y_test)


1/1 [==============================] - 0s 15ms/step - loss: 4.3464 - accuracy: 0.5000


[4.346424102783203, 0.5]

## Model 5

In [25]:
from keras.layers import BatchNormalization, LeakyReLU

model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(BatchNormalization())  # Add BatchNormalization after each Conv layer
model_10.add(Conv2D(64, (3, 3), activation='relu'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(BatchNormalization())
model_10.add(Conv2D(128, (3, 3), activation='relu'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(BatchNormalization())
model_10.add(Flatten())
model_10.add(Dense(256))
model_10.add(LeakyReLU(alpha=0.1))  # LeakyReLU activation
model_10.add(Dropout(0.5))
model_10.add(Dense(128))
model_10.add(LeakyReLU(alpha=0.1))
model_10.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model_10.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with adjusted architecture
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model_10.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=150,  # Train for more epochs
                    callbacks=es,
                    validation_data=(X_test, y_test))


Epoch 1/150
1/1 [==============================] - 1s 508ms/step - loss: 3.0810 - accuracy: 0.0952 - val_loss: 12.0135 - val_accuracy: 0.0625
Epoch 2/150
1/1 [==============================] - 0s 110ms/step - loss: 3.0917 - accuracy: 0.1746 - val_loss: 14.8459 - val_accuracy: 0.1250
Epoch 3/150
1/1 [==============================] - 0s 105ms/step - loss: 3.2399 - accuracy: 0.1270 - val_loss: 13.2046 - val_accuracy: 0.1250
Epoch 4/150
1/1 [==============================] - 0s 115ms/step - loss: 3.0123 - accuracy: 0.2222 - val_loss: 7.9506 - val_accuracy: 0.1250
Epoch 5/150
1/1 [==============================] - 0s 110ms/step - loss: 3.0556 - accuracy: 0.1905 - val_loss: 7.2680 - val_accuracy: 0.0625
Epoch 6/150
1/1 [==============================] - 0s 111ms/step - loss: 2.6237 - accuracy: 0.3651 - val_loss: 6.1577 - val_accuracy: 0.0625
Epoch 7/150
1/1 [==============================] - 0s 108ms/step - loss: 2.5636 - accuracy: 0.3016 - val_loss: 4.6323 - val_accuracy: 0.0625
Epoch 8/15

In [26]:
model_10.evaluate(X_test, y_test)


1/1 [==============================] - 0s 18ms/step - loss: 1.8204 - accuracy: 0.4375


[1.820360779762268, 0.4375]

# Video + Getty - 10 Classes (video_cropped_faces_1)

In [27]:
cropped_faces_directory = "../data_processing/raw_data/video_cropped_faces_1"


In [28]:
y, X = encode(cropped_faces_directory)
y_cat = to_cat(y)


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)


## Model 1

In [30]:
# Add a new classification head
layer = resnet.output
layer = GlobalAveragePooling2D()(layer)
predictions = Dense(y_train.shape[1], activation='softmax')(layer)

model_11 = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model_11.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

# Train the model
model_11.fit(X_train,
          y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[es]
         )


Epoch 1/10
13/13 [==============================] - 5s 238ms/step - loss: 2.2978 - accuracy: 0.3333 - val_loss: 500.6413 - val_accuracy: 0.0962
Epoch 2/10
13/13 [==============================] - 3s 207ms/step - loss: 1.5708 - accuracy: 0.4657 - val_loss: 230.2468 - val_accuracy: 0.0769
Epoch 3/10
13/13 [==============================] - 2s 182ms/step - loss: 1.1044 - accuracy: 0.6716 - val_loss: 66.1253 - val_accuracy: 0.0962
Epoch 4/10
13/13 [==============================] - 2s 180ms/step - loss: 1.0823 - accuracy: 0.7157 - val_loss: 63.3593 - val_accuracy: 0.1731
Epoch 5/10
13/13 [==============================] - 2s 183ms/step - loss: 0.6578 - accuracy: 0.8039 - val_loss: 124.3338 - val_accuracy: 0.1731
Epoch 6/10
13/13 [==============================] - 2s 184ms/step - loss: 0.4426 - accuracy: 0.8725 - val_loss: 174.7360 - val_accuracy: 0.1346
Epoch 7/10
13/13 [==============================] - 3s 194ms/step - loss: 0.4038 - accuracy: 0.8873 - val_loss: 222.5828 - val_accuracy: 0

In [31]:
model_11.evaluate(X_test, y_test)


2/2 [==============================] - 0s 89ms/step - loss: 102.5028 - accuracy: 0.2500


[102.50282287597656, 0.25]

## Model 2

In [32]:
# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add augmentation layers directly within the model
augmented = Conv2D(32, (3, 3), activation='relu')(input_layer)
augmented = MaxPooling2D((2, 2))(augmented)
# Add more layers or augmentation techniques as needed

# Rest of your model architecture
# Example:
augmented = Flatten()(augmented)
augmented = Dense(128, activation='relu')(augmented)
output_layer = Dense(y_train.shape[1], activation='softmax')(augmented)


model_12 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_12.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

datagen = ImageDataGenerator(
    rotation_range=40,       # Rotate images by a wider range of degrees
    width_shift_range=0.3,   # Shift images horizontally by a larger fraction of total width
    height_shift_range=0.3,  # Shift images vertically by a larger fraction of total height
    shear_range=0.3,         # Apply a larger shear-based transformation
    zoom_range=0.3,          # Zoom in or out of images more aggressively
    horizontal_flip=True,    # Flip images horizontally
    vertical_flip=True,      # Flip images vertically
    fill_mode='nearest'      # Fill points outside the boundaries using the nearest available value
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_12.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
8/8 [==============================] - 0s 16ms/step - loss: 918.5413 - accuracy: 0.0781
Epoch 2/100
8/8 [==============================] - 0s 16ms/step - loss: 316.7346 - accuracy: 0.1055
Epoch 3/100
8/8 [==============================] - 0s 16ms/step - loss: 95.3333 - accuracy: 0.1055
Epoch 4/100
8/8 [==============================] - 0s 16ms/step - loss: 42.5824 - accuracy: 0.0859
Epoch 5/100
8/8 [==============================] - 0s 16ms/step - loss: 23.1921 - accuracy: 0.1562
Epoch 6/100
8/8 [==============================] - 0s 16ms/step - loss: 11.2264 - accuracy: 0.1328
Epoch 7/100
8/8 [==============================] - 0s 16ms/step - loss: 5.6333 - accuracy: 0.1133
Epoch 8/100
8/8 [==============================] - 0s 16ms/step - loss: 4.0300 - accuracy: 0.1289
Epoch 9/100
8/8 [==============================] - 0s 16ms/step - loss: 3.5746 - accuracy: 0.1250
Epoch 10/100
8/8 [==============================] - 0s 16ms/step - loss: 3.2352 - accuracy: 0.1719
Epoch 11/10

In [33]:
model_12.evaluate(X_test, y_test)


2/2 [==============================] - 0s 4ms/step - loss: 3.1155 - accuracy: 0.1719


[3.1155338287353516, 0.171875]

## Model 3

In [34]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add convolutional layers with pooling
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

# Flatten the output before feeding into dense layers
flatten = Flatten()(pool2)

# Add dense layers with dropout for regularization
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)  # Dropout layer for regularization

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)  # Dropout layer for regularization

# Output layer with softmax activation for multi-class classification
output_layer = Dense(y_train.shape[1], activation='softmax')(dropout2)

# Create the model
model_13 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_13.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create an ImageDataGenerator with multiple augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Vary brightness
    channel_shift_range=50,        # Vary channel shifts
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_13.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)


Epoch 1/100
8/8 [==============================] - 0s 24ms/step - loss: 66.4934 - accuracy: 0.0742
Epoch 2/100
8/8 [==============================] - 0s 24ms/step - loss: 3.6263 - accuracy: 0.1016
Epoch 3/100
8/8 [==============================] - 0s 25ms/step - loss: 2.3825 - accuracy: 0.0820
Epoch 4/100
8/8 [==============================] - 0s 25ms/step - loss: 2.3290 - accuracy: 0.1016
Epoch 5/100
8/8 [==============================] - 0s 23ms/step - loss: 2.3017 - accuracy: 0.0820
Epoch 6/100
8/8 [==============================] - 0s 23ms/step - loss: 2.3126 - accuracy: 0.1094
Epoch 7/100
8/8 [==============================] - 0s 23ms/step - loss: 2.3029 - accuracy: 0.0859
Epoch 8/100
8/8 [==============================] - 0s 24ms/step - loss: 2.3026 - accuracy: 0.1172
Epoch 9/100
8/8 [==============================] - 0s 24ms/step - loss: 2.2967 - accuracy: 0.1055
Epoch 10/100
8/8 [==============================] - 0s 24ms/step - loss: 2.2875 - accuracy: 0.1055
Epoch 11/100
8/8 [

In [35]:
model_13.evaluate(X_test, y_test)


2/2 [==============================] - 0s 5ms/step - loss: 2.2528 - accuracy: 0.1094


[2.252798080444336, 0.109375]

## Model 4

In [36]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_14 = Sequential()
model_14.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_14.add(MaxPooling2D((2, 2)))
model_14.add(Conv2D(64, (3, 3), activation='relu'))
model_14.add(MaxPooling2D((2, 2)))
model_14.add(Conv2D(128, (3, 3), activation='relu'))
model_14.add(MaxPooling2D((2, 2)))
model_14.add(Flatten())
model_14.add(Dense(256, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_14.add(Dropout(0.5))  # Adding Dropout layer
model_14.add(Dense(128, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_14.add(Dense(y_train.shape[1], activation='softmax'))

# Using Adam optimizer with a reduced learning rate
optimizer = Adam(lr=0.0001)
model_14.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Adding more callbacks, e.g., ReduceLROnPlateau
callbacks = [EarlyStopping(patience=5, restore_best_weights=True),
             ReduceLROnPlateau(factor=0.1, patience=3)]

# Fit the model with increased complexity and callbacks
history = model_14.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=(X_test, y_test))


Epoch 1/100


/Users/yassinebouaine/.pyenv/versions/3.10.6/envs/dynamic-players-insights/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


16/16 [==============================] - 1s 23ms/step - loss: 17.1859 - accuracy: 0.1367 - val_loss: 7.5361 - val_accuracy: 0.1250 - lr: 0.0010
Epoch 2/100
16/16 [==============================] - 0s 19ms/step - loss: 7.3219 - accuracy: 0.0820 - val_loss: 7.0795 - val_accuracy: 0.0469 - lr: 0.0010
Epoch 3/100
16/16 [==============================] - 0s 18ms/step - loss: 6.9548 - accuracy: 0.1211 - val_loss: 6.8261 - val_accuracy: 0.1562 - lr: 0.0010
Epoch 4/100
16/16 [==============================] - 0s 20ms/step - loss: 6.7022 - accuracy: 0.1484 - val_loss: 6.5906 - val_accuracy: 0.1250 - lr: 0.0010
Epoch 5/100
16/16 [==============================] - 0s 18ms/step - loss: 6.4862 - accuracy: 0.1211 - val_loss: 6.4014 - val_accuracy: 0.1094 - lr: 0.0010
Epoch 6/100
16/16 [==============================] - 0s 18ms/step - loss: 6.3101 - accuracy: 0.1133 - val_loss: 6.2232 - val_accuracy: 0.0625 - lr: 0.0010
Epoch 7/100
16/16 [==============================] - 0s 18ms/step - loss: 6.1255 

In [37]:
model_14.evaluate(X_test, y_test)


2/2 [==============================] - 0s 9ms/step - loss: 2.8314 - accuracy: 0.4531


[2.8313820362091064, 0.453125]

## Model 5

In [38]:
from keras.layers import BatchNormalization, LeakyReLU

model_15 = Sequential()
model_15.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_15.add(MaxPooling2D((2, 2)))
model_15.add(BatchNormalization())  # Add BatchNormalization after each Conv layer
model_15.add(Conv2D(64, (3, 3), activation='relu'))
model_15.add(MaxPooling2D((2, 2)))
model_15.add(BatchNormalization())
model_15.add(Conv2D(128, (3, 3), activation='relu'))
model_15.add(MaxPooling2D((2, 2)))
model_15.add(BatchNormalization())
model_15.add(Flatten())
model_15.add(Dense(256))
model_15.add(LeakyReLU(alpha=0.1))  # LeakyReLU activation
model_15.add(Dropout(0.5))
model_15.add(Dense(128))
model_15.add(LeakyReLU(alpha=0.1))
model_15.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model_15.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with adjusted architecture
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model_15.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=150,  # Train for more epochs
                    callbacks=es,
                    validation_data=(X_test, y_test))


Epoch 1/150
4/4 [==============================] - 1s 161ms/step - loss: 3.3672 - accuracy: 0.1094 - val_loss: 6.9599 - val_accuracy: 0.0938
Epoch 2/150
4/4 [==============================] - 0s 61ms/step - loss: 3.3915 - accuracy: 0.1211 - val_loss: 7.9883 - val_accuracy: 0.1094
Epoch 3/150
4/4 [==============================] - 0s 63ms/step - loss: 2.8884 - accuracy: 0.1797 - val_loss: 5.3758 - val_accuracy: 0.1406
Epoch 4/150
4/4 [==============================] - 0s 70ms/step - loss: 3.0495 - accuracy: 0.1641 - val_loss: 3.2933 - val_accuracy: 0.1250
Epoch 5/150
4/4 [==============================] - 0s 62ms/step - loss: 2.8453 - accuracy: 0.1719 - val_loss: 5.7002 - val_accuracy: 0.0938
Epoch 6/150
4/4 [==============================] - 0s 68ms/step - loss: 2.6154 - accuracy: 0.1523 - val_loss: 3.5932 - val_accuracy: 0.1406
Epoch 7/150
4/4 [==============================] - 0s 59ms/step - loss: 2.8588 - accuracy: 0.1875 - val_loss: 3.2351 - val_accuracy: 0.0938
Epoch 8/150
4/4 [==

In [39]:
model_15.evaluate(X_test, y_test)


2/2 [==============================] - 0s 11ms/step - loss: 1.9127 - accuracy: 0.3125


[1.9126980304718018, 0.3125]

# Video + Getty - 5 Classes (video_cropped_faces_2)

In [40]:
cropped_faces_directory = "../data_processing/raw_data/video_cropped_faces_2"

y, X = encode(cropped_faces_directory)
y_cat = to_cat(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)


## Model 1

In [41]:
# Add a new classification head
layer = resnet.output
layer = GlobalAveragePooling2D()(layer)
predictions = Dense(y_train.shape[1], activation='softmax')(layer)

model_16 = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model_16.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

# Train the model
model_16.fit(X_train,
          y_train,
          epochs=10,
          batch_size=16,
          validation_split=0.2,
          callbacks=[es]
         )

model_16.evaluate(X_test, y_test)


Epoch 1/10
8/8 [==============================] - 4s 246ms/step - loss: 1.3650 - accuracy: 0.5948 - val_loss: 2921.5955 - val_accuracy: 0.1034
Epoch 2/10
8/8 [==============================] - 1s 173ms/step - loss: 0.7080 - accuracy: 0.7500 - val_loss: 1087.4312 - val_accuracy: 0.1034
Epoch 3/10
8/8 [==============================] - 1s 172ms/step - loss: 0.4975 - accuracy: 0.8362 - val_loss: 77.4192 - val_accuracy: 0.0690
Epoch 4/10
8/8 [==============================] - 1s 182ms/step - loss: 0.3822 - accuracy: 0.8707 - val_loss: 34.5097 - val_accuracy: 0.1034
Epoch 5/10
8/8 [==============================] - 1s 172ms/step - loss: 0.4215 - accuracy: 0.8707 - val_loss: 168.5349 - val_accuracy: 0.0345
Epoch 6/10
8/8 [==============================] - 1s 178ms/step - loss: 0.3427 - accuracy: 0.8966 - val_loss: 315.2168 - val_accuracy: 0.1724
Epoch 7/10
2/2 [==============================] - 0s 29ms/step - loss: 20.9034 - accuracy: 0.1892


[20.903432846069336, 0.18918919563293457]

## Model 2

In [42]:
# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add augmentation layers directly within the model
augmented = Conv2D(32, (3, 3), activation='relu')(input_layer)
augmented = MaxPooling2D((2, 2))(augmented)
# Add more layers or augmentation techniques as needed

# Rest of your model architecture
# Example:
augmented = Flatten()(augmented)
augmented = Dense(128, activation='relu')(augmented)
output_layer = Dense(y_train.shape[1], activation='softmax')(augmented)


model_17 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_17.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
es = EarlyStopping(patience=3,
                   restore_best_weights=True)

datagen = ImageDataGenerator(
    rotation_range=40,       # Rotate images by a wider range of degrees
    width_shift_range=0.3,   # Shift images horizontally by a larger fraction of total width
    height_shift_range=0.3,  # Shift images vertically by a larger fraction of total height
    shear_range=0.3,         # Apply a larger shear-based transformation
    zoom_range=0.3,          # Zoom in or out of images more aggressively
    horizontal_flip=True,    # Flip images horizontally
    vertical_flip=True,      # Flip images vertically
    fill_mode='nearest'      # Fill points outside the boundaries using the nearest available value
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_17.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)

model_17.evaluate(X_test, y_test)


Epoch 1/100
5/5 [==============================] - 0s 15ms/step - loss: 1130.9338 - accuracy: 0.1310
Epoch 2/100
5/5 [==============================] - 0s 16ms/step - loss: 715.9363 - accuracy: 0.1862
Epoch 3/100
5/5 [==============================] - 0s 17ms/step - loss: 171.3322 - accuracy: 0.1931
Epoch 4/100
5/5 [==============================] - 0s 15ms/step - loss: 122.4031 - accuracy: 0.2276
Epoch 5/100
5/5 [==============================] - 0s 14ms/step - loss: 82.0305 - accuracy: 0.2276
Epoch 6/100
5/5 [==============================] - 0s 14ms/step - loss: 36.9867 - accuracy: 0.2690
Epoch 7/100
5/5 [==============================] - 0s 14ms/step - loss: 12.4112 - accuracy: 0.3586
Epoch 8/100
5/5 [==============================] - 0s 14ms/step - loss: 10.5632 - accuracy: 0.2138
Epoch 9/100
5/5 [==============================] - 0s 16ms/step - loss: 8.9128 - accuracy: 0.2207
Epoch 10/100
5/5 [==============================] - 0s 14ms/step - loss: 5.8153 - accuracy: 0.2138
Epoch 

[2.180905818939209, 0.37837839126586914]

## Model 3

In [43]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape for your images
input_shape = X_train.shape[1:]

# Define a Sequential model or Functional API model
input_layer = Input(shape=input_shape)

# Add convolutional layers with pooling
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

# Flatten the output before feeding into dense layers
flatten = Flatten()(pool2)

# Add dense layers with dropout for regularization
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.5)(dense1)  # Dropout layer for regularization

dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)  # Dropout layer for regularization

# Output layer with softmax activation for multi-class classification
output_layer = Dense(y_train.shape[1], activation='softmax')(dropout2)

# Create the model
model_18 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_18.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Create an ImageDataGenerator with multiple augmentation techniques
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Vary brightness
    channel_shift_range=50,        # Vary channel shifts
    fill_mode='nearest'
)

# Fit the ImageDataGenerator to your data
datagen.fit(X_train)

# Example of using the ImageDataGenerator during model training
model_18.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100)

model_18.evaluate(X_test, y_test)


Epoch 1/100
5/5 [==============================] - 0s 23ms/step - loss: 104.2619 - accuracy: 0.2069
Epoch 2/100
5/5 [==============================] - 0s 24ms/step - loss: 16.2475 - accuracy: 0.2000
Epoch 3/100
5/5 [==============================] - 0s 20ms/step - loss: 2.9138 - accuracy: 0.1655
Epoch 4/100
5/5 [==============================] - 0s 21ms/step - loss: 1.7119 - accuracy: 0.2690
Epoch 5/100
5/5 [==============================] - 0s 24ms/step - loss: 1.6351 - accuracy: 0.2138
Epoch 6/100
5/5 [==============================] - 0s 21ms/step - loss: 1.6743 - accuracy: 0.1793
Epoch 7/100
5/5 [==============================] - 0s 22ms/step - loss: 1.6898 - accuracy: 0.2414
Epoch 8/100
5/5 [==============================] - 0s 21ms/step - loss: 1.6279 - accuracy: 0.2138
Epoch 9/100
5/5 [==============================] - 0s 22ms/step - loss: 1.5850 - accuracy: 0.2483
Epoch 10/100
5/5 [==============================] - 0s 20ms/step - loss: 1.6030 - accuracy: 0.1862
Epoch 11/100
5/5

[1.5140997171401978, 0.2702702581882477]

## Model 4

In [44]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

model_19 = Sequential()
model_19.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_19.add(MaxPooling2D((2, 2)))
model_19.add(Conv2D(64, (3, 3), activation='relu'))
model_19.add(MaxPooling2D((2, 2)))
model_19.add(Conv2D(128, (3, 3), activation='relu'))
model_19.add(MaxPooling2D((2, 2)))
model_19.add(Flatten())
model_19.add(Dense(256, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_19.add(Dropout(0.5))  # Adding Dropout layer
model_19.add(Dense(128, activation='relu', kernel_regularizer='l2'))  # Adding L2 regularization
model_19.add(Dense(y_train.shape[1], activation='softmax'))

# Using Adam optimizer with a reduced learning rate
optimizer = Adam(lr=0.0001)
model_19.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Adding more callbacks, e.g., ReduceLROnPlateau
callbacks = [EarlyStopping(patience=5, restore_best_weights=True),
             ReduceLROnPlateau(factor=0.1, patience=3)]

# Fit the model with increased complexity and callbacks
history = model_19.fit(datagen.flow(X_train, y_train, batch_size=16),
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=(X_test, y_test))

model_19.evaluate(X_test, y_test)


Epoch 1/100


/Users/yassinebouaine/.pyenv/versions/3.10.6/envs/dynamic-players-insights/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


10/10 [==============================] - 1s 29ms/step - loss: 40.9099 - accuracy: 0.2138 - val_loss: 14.8091 - val_accuracy: 0.2162 - lr: 0.0010
Epoch 2/100
10/10 [==============================] - 0s 17ms/step - loss: 9.4812 - accuracy: 0.2000 - val_loss: 5.8835 - val_accuracy: 0.3784 - lr: 0.0010
Epoch 3/100
10/10 [==============================] - 0s 18ms/step - loss: 5.8014 - accuracy: 0.1793 - val_loss: 5.4768 - val_accuracy: 0.2162 - lr: 0.0010
Epoch 4/100
10/10 [==============================] - 0s 17ms/step - loss: 5.4083 - accuracy: 0.2621 - val_loss: 5.3563 - val_accuracy: 0.1351 - lr: 0.0010
Epoch 5/100
10/10 [==============================] - 0s 18ms/step - loss: 5.2932 - accuracy: 0.1931 - val_loss: 5.2414 - val_accuracy: 0.2432 - lr: 0.0010
Epoch 6/100
10/10 [==============================] - 0s 19ms/step - loss: 5.2058 - accuracy: 0.2276 - val_loss: 5.1465 - val_accuracy: 0.1892 - lr: 0.0010
Epoch 7/100
10/10 [==============================] - 0s 17ms/step - loss: 5.1006

[3.790188789367676, 0.4864864945411682]

## Model 5

In [45]:
from keras.layers import BatchNormalization, LeakyReLU

model_20 = Sequential()
model_20.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model_20.add(MaxPooling2D((2, 2)))
model_20.add(BatchNormalization())  # Add BatchNormalization after each Conv layer
model_20.add(Conv2D(64, (3, 3), activation='relu'))
model_20.add(MaxPooling2D((2, 2)))
model_20.add(BatchNormalization())
model_20.add(Conv2D(128, (3, 3), activation='relu'))
model_20.add(MaxPooling2D((2, 2)))
model_20.add(BatchNormalization())
model_20.add(Flatten())
model_20.add(Dense(256))
model_20.add(LeakyReLU(alpha=0.1))  # LeakyReLU activation
model_20.add(Dropout(0.5))
model_20.add(Dense(128))
model_20.add(LeakyReLU(alpha=0.1))
model_20.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model_20.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with adjusted architecture
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model_20.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=150,  # Train for more epochs
                    callbacks=es,
                    validation_data=(X_test, y_test))


model_20.evaluate(X_test, y_test)


Epoch 1/150
3/3 [==============================] - 1s 120ms/step - loss: 3.0575 - accuracy: 0.1862 - val_loss: 6.6435 - val_accuracy: 0.2162
Epoch 2/150
3/3 [==============================] - 0s 46ms/step - loss: 2.4267 - accuracy: 0.2276 - val_loss: 3.7690 - val_accuracy: 0.1622
Epoch 3/150
3/3 [==============================] - 0s 43ms/step - loss: 2.7360 - accuracy: 0.1931 - val_loss: 7.7884 - val_accuracy: 0.1622
Epoch 4/150
3/3 [==============================] - 0s 55ms/step - loss: 2.1700 - accuracy: 0.3241 - val_loss: 7.9802 - val_accuracy: 0.1351
Epoch 5/150
3/3 [==============================] - 0s 55ms/step - loss: 2.6567 - accuracy: 0.2552 - val_loss: 6.1181 - val_accuracy: 0.1622
Epoch 6/150
3/3 [==============================] - 0s 72ms/step - loss: 2.3267 - accuracy: 0.2828 - val_loss: 6.1946 - val_accuracy: 0.1351
Epoch 7/150
3/3 [==============================] - 0s 74ms/step - loss: 2.6628 - accuracy: 0.2759 - val_loss: 4.7975 - val_accuracy: 0.1622
Epoch 8/150
3/3 [==

[1.3636188507080078, 0.4324324429035187]

# Testing on video

In [46]:
# import cv2
# import os
# import pandas as pd
# from PIL import Image
# import numpy as np
# import face_recognition


In [47]:
# def crop_video(input_directory, output_directory) :
#      # Create the output directory if it doesn't exist
#     if not os.path.exists(output_directory):
#         os.makedirs(output_directory)

#     for filename in os.listdir(input_directory):
#         if filename.endswith(".jpg") or filename.endswith(".jpeg"):
#             file_path = os.path.join(input_directory, filename)

#             # Load the image using face_recognition library
#             image = face_recognition.load_image_file(file_path)
#             face_locations = face_recognition.face_locations(image)

#             # Check if a face is detected
#             if len(face_locations) > 0:
#                 # Crop and save each face detected
#                 for idx, face_location in enumerate(face_locations):
#                     top, right, bottom, left = face_location
#                     face_image = image[top:bottom, left:right]
#                     pil_image = Image.fromarray(face_image)
#                     cropped_file_name = f"{os.path.splitext(filename)[0]}_face_{idx + 1}.jpg"
#                     cropped_file_path = os.path.join(output_directory, cropped_file_name)
#                     pil_image.save(cropped_file_path)
#                     print(f"Face cropped and saved: {cropped_file_name}")
#             else:
#                 print(f"No face detected in {filename}, skipping...")


In [48]:
# import os
# import cv2
# import face_recognition
# from PIL import Image

# def save_frames_with_face(video_path, dir_path, basename, ext='jpg', frames_per_second=1):
#     cap = cv2.VideoCapture(video_path)

#     if not cap.isOpened():
#         return

#     os.makedirs(dir_path, exist_ok=True)
#     base_path = os.path.join(dir_path, basename)

#     fps = cap.get(cv2.CAP_PROP_FPS)
#     frame_count = 0

#     frame_interval = int(fps / frames_per_second) if frames_per_second > 0 else 1

#     while True:
#         ret, frame = cap.read()
#         if ret:
#             if frame_count % frame_interval == 0:
#                 # Convert frame to RGB for face recognition library
#                 frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#                 # Detect face locations in the frame
#                 face_locations = face_recognition.face_locations(frame_rgb)

#                 if len(face_locations) > 0:
#                     # Extract the first face detected (you can modify this logic)
#                     top, right, bottom, left = face_locations[0]
#                     face_image = frame[top:bottom, left:right]  # Crop the face
#                     pil_image = Image.fromarray(face_image)

#                     # Save the frame with the detected face
#                     frame_path = f"{base_path}_frame_{frame_count}_cropped_face.{ext}"
#                     pil_image.save(frame_path)
#                     print(f"Face detected and saved: {frame_path}")

#             frame_count += 1
#         else:
#             break

#     cap.release()


In [49]:
# video_path = "/Users/yassinebouaine/Downloads/demo_1.mov"
# dir_path = "../data_processing/raw_data/prediction"
# basename = "prediction"


In [50]:
# cropped_image = save_frames_with_face(video_path, dir_path, basename, ext='jpg', frames_per_second=1)


In [55]:
import numpy as np


In [51]:
y_new, X_new = encode("../data_processing/raw_data/prediction/")


In [52]:
X_new.shape


(1, 64, 64, 3)

In [56]:
pred = model_9.predict(X_new)
pred


1/1 [==============================] - 0s 46ms/step


array([[0.05658385, 0.05945316, 0.14116952, 0.00836595, 0.00451716,
        0.00508426, 0.00423702, 0.05495046, 0.07961128, 0.1930185 ,
        0.3930089 ]], dtype=float32)

In [57]:
predicted_index = np.argmax(pred)
predicted_index


10

In [59]:
predicted_label = y_cat[predicted_index]
predicted_label


array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [61]:
player_name = y[np.where((y_cat == predicted_label).all(axis=1))[0][0]]
player_name


'benzema'